In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

## November Predictions
At the time of starting this project. The NPD had not yet released sales data for November 2018. Here are predictions using my trained model.

In [2]:
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

from pytrends.request import TrendReq

pytrends = TrendReq(hl='en-US', tz=360)

def bestmatch(game):
    match = game
    match_n = game
    try:
        googsuggest = pytrends.suggestions(game)
    except:
        googsuggest = []
    s = 0
    for i in range(0,len(googsuggest)):
        name = googsuggest[i]['title']
        s_type = googsuggest[i]['type']
        mid = googsuggest[i]['mid']
        sim = similar(name.lower(),game.lower())
        if sim > s and ('Video game' in s_type or 'video game' in s_type or 'Console game' in s_type or 'Computer game' in s_type or 'Online game' in s_type or 'Musical game' in s_type or 'Survival game' in s_type):
            s = sim
            match = mid
            match_n = name
    if s < 0.925:
        match = game 
    return [match,match_n]

Adding games titles not included in pervious data:

In [3]:
newgamelist = ["Spyro Reignited Trilogy", "Pokemon: Let's Go Pikachu", "Pokemon: Let's Go Eevee", "Battlefield V", "World of Final Fantasy Maxima"
              , "The Quiet Man", "Deracine", "Overkill's The Walking Dead", "Ride 3", "Tetris Effect", "Hitman 2", "SNK 40th Anniversary Collection", "Underworld Ascendant"
              , "Civilization VI", "Warframe", "Wreckfest", "Darksiders 3", "Artifact", "Just Cause 4", "Monster Boy and the Cursed Kingdom", "Persona 3: Dancing in Moonlight"
              , "Persona 4: Dancing All Night", "Persona 5: Dancing in Starlight", "Super Smash Bros Ultimate", "Earth Defense Force 5", "Firewatch", "Katamari Damacy Reroll"
              ,"Just Dance 2019", "Spider-Man"]

In [4]:
newgamematches = []
for i in range(0,len(newgamelist)):
    [match,match_n] = bestmatch(newgamelist[i])
    newgamematches.append([newgamelist[i],match_n,match])

In [5]:


from time import sleep
import timeit
def req(kw_list, slp, tf):
    while True:
        sleep(slp)
        try:
            pytrends.build_payload(kw_list, cat=0, timeframe=tf, geo='US', gprop='')
            trends = pytrends.interest_over_time()
            break
        except:
            print("wait...")
            sleep(60)
    return trends



In [6]:
kw_list = ["Halo 4"]
alltrends1 = req(kw_list, slp=0.1, tf = '2008-12-22 2013-12-15')
alltrends1.loc[:,"Halo 4"] *= 88/100 #rescaled to Dark Souls III

kw_list = ['/m/013f7bmb'] #Dark Souls III
alltrends2 = req(kw_list, slp=0.1, tf = '2013-12-22 2018-12-16')
alltrends2.columns = ['Dark Souls III', 'isPartial'] 

for i in range(0,len(newgamematches)):
    [game,match_n,match] = newgamematches[i]
    if game == "Halo 4" or game =='Dark Souls III':
        print("skip")
    else:
        kw_list1 = ["Halo 4",match]
        trends1 = req(kw_list1, slp=0.1, tf = '2008-12-22 2013-12-15')
        kw_list2 = ['/m/013f7bmb',match]
        trends2 = req(kw_list2, slp=0.1, tf = '2013-12-22 2018-12-16')
        mavalu1 = trends1.max(0)[1]
        mavalu2 = trends2.max(0)[1]

        Halomax = trends1.max(0)[0]
        trends1.iloc[:,1] *= 88/Halomax
        trends1.columns = ['Halo 4', game, 'isPartial'] 
        alltrends1 = pd.DataFrame.join(alltrends1.copy(),
                trends1[game],
                on='date',
                how = 'left')

        Darkmax = trends2.max(0)[0]
        trends2.iloc[:,1] *= 100/Darkmax
        trends2.columns = ['/m/013f7bmb', game, 'isPartial'] 
        alltrends2 = pd.DataFrame.join(alltrends2.copy(),
                trends2[game],
                on='date',
                how = 'left')

alltrends2.drop("Dark Souls III", axis = 1)
alltrends1.drop("Halo 4", axis = 1)
alltrends = pd.concat([alltrends1,alltrends2], sort = True)

In [7]:
def xmonthsago (xmonths, date,datelist): #pulls out dates from datelist from x months before date
    #YYYY-MM-DD
    thedays = []
    year_i = int(date[:4])
    month_i = int(date[5:7])
    m = month_i - xmonths
    y = year_i
    while m<1:
        m = m+12
        y = y -1
    if m < 10:
        str_m = '0'+str(m)
    else: 
        str_m = str(m)
    ym = str(y)+'-'+str_m
    for i in range(0,len(datelist)):
        if ym in datelist[i]:
            thedays.append(datelist[i])
    return thedays

datelist = list(alltrends.index.values)
datelist_str = [str(date)[0:10] for date in datelist]
alltrends.index = datelist_str

In [8]:
def getfeature (data, game, thedays):
    x = data.filter(items=thedays,axis=0).max()[game]   
    return x

In [9]:
import pickle
with open('alltrends_123.pickle', 'rb') as handle:
    alltrends_12 = pickle.load(handle)
    
with open('gamelist.pickle', 'rb') as handle:
    oldgamelist = pickle.load(handle)
datelist_old = list(alltrends_12.index.values)

In [10]:
X = np.zeros([1,len(newgamelist)+len(oldgamelist),6]) #month, game, features
date_format = "%Y-%m-%d"


thedays1 = xmonthsago(1, "2018-12-01", datelist_old)
thedays2 = xmonthsago(2, "2018-12-01", datelist_old)
thedays3 = xmonthsago(3, "2018-12-01", datelist_old)
    
for j in range(0,len(oldgamelist)):
    maxdate = alltrends_12[oldgamelist[j]].idxmax()[:10] 
    x1 = (datetime.strptime("2018-12-01", date_format) - datetime.strptime(maxdate, date_format)).days
    if x1 < 0:
        x2 = 0
    else:
        x2 = alltrends_12.max()[oldgamelist[j]]
    X[0,j,0] = x1
    X[0,j,1] = x2
    X[0,j,2] = getfeature(alltrends_12,oldgamelist[j],thedays1)
    X[0,j,3] = getfeature(alltrends_12,oldgamelist[j],thedays2)
    X[0,j,4] = getfeature(alltrends_12,oldgamelist[j],thedays3)
    if x1 >= 5 and x1 <= 55: #Did the game come out recently? (but not too recently)
        X[0,j,5] = 1
    else: 
        X[0,j,5] = -1

thedays1 = xmonthsago(1, "2018-12-01", datelist_str)
thedays2 = xmonthsago(2, "2018-12-01", datelist_str)
thedays3 = xmonthsago(3, "2018-12-01", datelist_str)

for j in range(0,len(newgamelist)):
    maxdate = str(alltrends[newgamelist[j]].idxmax())[:10] 
    x1 = (datetime.strptime("2018-12-01", date_format) - datetime.strptime(maxdate, date_format)).days
    if x1 < 0:
        x2 = 0
    else:
        x2 = alltrends.max()[newgamelist[j]]
    X[0,len(oldgamelist)+j,0] = x1
    X[0,len(oldgamelist)+j,1] = x2
    X[0,len(oldgamelist)+j,2] = getfeature(alltrends,newgamelist[j],thedays1)
    X[0,len(oldgamelist)+j,3] = getfeature(alltrends,newgamelist[j],thedays2)
    X[0,len(oldgamelist)+j,4] = getfeature(alltrends,newgamelist[j],thedays3)
    if x1 >= 5 and x1 <= 55: #Did the game come out recently? (but not too recently)
        X[0,j,5] = 1
    else: 
        X[0,j,5] = -1

In [11]:
(l,m,n) = X.shape
X_new = []
index_values = [] #saves which indices are saved

for j in range(0,m): #1st game
    if X[0,j,0]>-1: #checking if game has come out yet
        for k in range(j+1,m): #looping over 2nd game
            if X[0,k,0]>-1:
                if list(X[0,j,2:5]) == [0,0,0] and list(X[0,k,2:5]) == [0,0,0]: #not worth comparing if approx 0 searches in last 3 months
                    pass
                else:
                    x_pair = list(X[0,j] - X[0,k])
                    X_new.append(x_pair)
                    index_values.append([0,j,k])

In [12]:
with open('clf.pickle', 'rb') as handle:
    clf = pickle.load(handle)
    
Nov_predictions = clf.predict(X_new)

In [13]:
def BTmodel (indices, Y, n, loops): #seems to rank known ones most accurately when looped multiple times
    p = 100*np.ones(n)/n
    for k in range(0,loops):
        for j in range(0,len(Y)):
            game1_index = indices[j][1]
            game2_index = indices[j][2]
            if Y[j] == -1: #game 1 wins
                p[game1_index] = (p[game2_index]+p[game1_index])
            else: #game 2 wins
                p[game2_index] = (p[game2_index]+p[game1_index])
            p = 100*p/p.sum() #renormalize
    return p

#def BTmodel (indices, Y, n, loops): #seems to do better when looped once
#    p = 100*np.ones(n)/n
#    for k in range(0,loops):
#        j = 0
#        for i in range(0,n):
#            W = 0
#            s = 0
#            game1_index = i
#            while game1_index == i and j < len(Y):
#                game2_index = indices[j][2]
#                if Y[j] == -1:
#                    W = W+1
#                    s = s+ 1/(p[game2_index]+p[game1_index])
#                else:
#                    s = s+ 1/(p[game2_index]+p[game1_index])
#                j = j+1
#                try:
#                    game1_index = indices[j][1]
#                except:
#                    pass
#                if s != 0:
#                    p[i] = W/s
#                    p = 100*p/p.sum() #renormalize
#    return p

In [14]:
p = BTmodel(index_values,Nov_predictions, len(oldgamelist+newgamelist),12)

In [15]:
biggamelist = oldgamelist + newgamelist
dtype = [('game','S35'),('p',float)]
q = []
for i in range(0,len(p)):
    q.append((biggamelist[i],p[i]))
qarray = np.array(q,dtype = dtype)
qsort = np.sort(qarray, order = 'p')

In [16]:
list(qsort[::-1][0:21])

[(b'Spider-Man', 43.1525772),
 (b'Warframe', 9.6035746),
 (b'Daylight', 6.50067294),
 (b'Fallout 76', 6.43306809),
 (b'Spyro Reignited Trilogy', 3.69693204),
 (b"Pokemon: Let's Go Pikachu", 3.52210811),
 (b"Pokemon: Let's Go Eevee", 3.33866293),
 (b'Battlefield V', 3.16382886),
 (b"Overkill's The Walking Dead", 2.97559406),
 (b'Tetris Effect', 2.77366122),
 (b'Hitman 2', 2.57172831),
 (b'Civilization VI', 2.35708236),
 (b'Darksiders 3', 2.30565329),
 (b'Artifact', 1.69248419),
 (b'Just Dance 2019', 1.55831701),
 (b'Darksiders III', 0.95941306),
 (b'Call of Cthulhu', 0.81728697),
 (b'Fe', 0.76110393),
 (b'Extinction', 0.1907546),
 (b'Ultra Street Fighter II: The Final ', 0.18715249),
 (b'1-2-Switch', 0.18597728)]

Compare this with the now released actual rankings:
1. Red Dead Redemption 2
2. Call of Duty: Black Ops 4
3. Battlefield V
4. Fallout 76
5. Pokemon: Let's Go Pikachu
6. Pokemon: Let's Go Eevee
7. NBA 2K19
8. Madden NFL 19
9. Spyro Reignited Trilogy
10. FIFA 19
11. NBA 2K19
12. Super Mario Party
13. Spider-Man
14. Mario Kart 8
15. WWE 2k19
16. God of War
17. Shadow of the Tomb Raider
18. Just Dance 2019
19. Grand Theft Auto V
20. Forza Horizon 4